In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

In [23]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                              'Agencia_ID': 'Sales_Depot_ID',
                              'Canal_ID': 'Sales_Channel_ID',
                              'Ruta_SAK': 'Route_ID',
                              'Cliente_ID': 'Client_ID',
                              'Venta_uni_hoy': 'Sales_unit_this_week',
                              'Venta_hoy': 'Sales_this_week',
                              'Dev_uni_proxima': 'Returns_unit_next_week',
                              'Dev_proxima': 'Returns_next_week',
                              'Demanda_uni_equil': 'adjusted_demand',
                              'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                            'Agencia_ID': 'Sales_Depot_ID',
                            'Canal_ID': 'Sales_Channel_ID',
                            'Ruta_SAK': 'Route_ID',
                            'Cliente_ID': 'Client_ID',
                            'Venta_uni_hoy': 'Sales_unit_this_week',
                            'Venta_hoy': 'Sales_this_week',
                            'Dev_uni_proxima': 'Returns_unit_next_week',
                            'Dev_proxima': 'Returns_next_week',
                            'Demanda_uni_equil': 'adjusted_demand',
                            'Producto_ID': 'Product_ID'})



#set a unique id for each sales depot id, sales channel id, route id, client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

#set a combined client ID, consisting of a unique sales depot ID, sales channel ID, route ID, and client ID
combined_df['ccid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']).ngroup()

#set a combined product ID, consisting of a unique sales depot ID, sales channel ID, route ID, and product ID
combined_df['cpid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train['adjusted_demand'] = np.log1p(train['adjusted_demand'])
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

In [24]:
#create a dataframe of aggregate statistics for each client
testagg = train[train['Week_num'] <= 8].sort_values(by=['ccid']).groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

#create a dataframe of aggregate statistics for each product
testagg = train[train['Week_num'] <= 8].sort_values(by=['cpid']).groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']
product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg

In [42]:
len(train[train['Client_ID'] == 26]['Product_ID'].unique())

51

In [51]:
train[(train['Client_ID'] == 65) & (train['Week_num'] <= 8)]['adjusted_demand'].sum()

np.float64(2251.1750916392903)

In [ ]:
# get ccid, cpid means, medians and cpid median percentage into training data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_mean'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_median'] = train['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_mean'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median'] = train['cpid'].map(pidmapping)

# pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['median_pct'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
# train['cpid_median_pct'] = train['cpid'].map(pidmapping).round(3)


# get total number of different products purchased by each client
# get total number of clients 

del cidmapping, pidmapping

In [27]:
# create training data based on ID in order to use lagged adjusted demand
trainIDdf = pd.DataFrame()

trainIDdf = train[['ID', 'Client_ID', 'Product_ID', 'ccid_mean', 'cpid_mean']].drop_duplicates(subset='ID', keep='first').reset_index(drop=True)

# get adjusted demand for the week for each ID 
for j in [3,4,5,6,7,8,9]:
    wkmap = pd.Series(train[train['Week_num'] == j].set_index('ID')['adjusted_demand'], index=train['ID'].unique()).to_dict()
    trainIDdf[f'Wk_{j}_dem'] = trainIDdf['ID'].map(wkmap)

trainIDdf['ID'] = trainIDdf['ID'].astype('category')
trainIDdf['Client_ID'] = trainIDdf['Client_ID'].astype('category')
trainIDdf['Product_ID'] = trainIDdf['Product_ID'].astype('category')

trainIDdf.head()

,ID,Client_ID,Product_ID,ccid_mean,cpid_mean,Wk_3_dem,Wk_4_dem,Wk_5_dem,Wk_6_dem,Wk_7_dem,Wk_8_dem,Wk_9_dem
0,0,15766,325,1.72,0.98,NaN,0.693147,NaN,NaN,NaN,NaN,NaN
1,1,15766,328,1.72,0.99,NaN,0.693147,NaN,NaN,NaN,NaN,NaN
2,2,15766,1212,1.72,1.19,1.386294,1.609438,1.791759,NaN,NaN,1.609438,0.693147
3,3,15766,1216,1.72,1.09,1.609438,1.098612,1.386294,0.693147,1.098612,1.791759,NaN
4,4,15766,1220,1.72,1.32,NaN,NaN,1.386294,NaN,NaN,0.693147,NaN


In [30]:
X_train = trainIDdf.iloc[:, 1:-1]

trainIDdf['Wk_9_dem'].fillna(trainIDdf['ccid_mean'], inplace=True)
y_train = trainIDdf.iloc[:, -1]

X_train.drop(['Wk_3_dem', 'Wk_4_dem','Wk_5_dem'], axis=1, inplace=True)
X_train.rename(columns={'Wk_6_dem': 'lag_3', 'Wk_7_dem': 'lag_2', 'Wk_8_dem': 'lag_1'}, inplace=True)

X_train.head()

C:\Users\eblac\AppData\Local\Temp\ipykernel_11408\3272150173.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainIDdf['Wk_9_dem'].fillna(trainIDdf['ccid_mean'], inplace=True)


,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1
0,15766,325,1.72,0.98,NaN,NaN,NaN
1,15766,328,1.72,0.99,NaN,NaN,NaN
2,15766,1212,1.72,1.19,NaN,NaN,1.609438
3,15766,1216,1.72,1.09,0.693147,1.098612,1.791759
4,15766,1220,1.72,1.32,NaN,NaN,0.693147


In [33]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

model = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=100)

In [34]:
# get ccid, cpid means and medians into test data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_mean'] = test['ccid'].map(cidmapping)

# cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
# test['ccid_median'] = test['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_mean'] = test['cpid'].map(pidmapping)

# pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
# test['cpid_median'] = test['cpid'].map(pidmapping)

del cidmapping, pidmapping

test = test[['id', 'ID', 'Week_num', 'Client_ID', 'Product_ID', 'ccid_mean', 'cpid_mean']].sort_values(by='id')


# get adjusted demand from previous weeks
lagmap = pd.Series(trainIDdf[['ID', 'Wk_7_dem']].set_index('ID')['Wk_7_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_3'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_8_dem']].set_index('ID')['Wk_8_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_2'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_9_dem']].set_index('ID')['Wk_9_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_1'] = test['ID'].map(lagmap)

del lagmap


test['Client_ID'] = test['Client_ID'].astype('category')
test['Product_ID'] = test['Product_ID'].astype('category')

test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1
6558101,0,25973294,11,4639078,35305,1.21,1.67,NaN,NaN,1.609438
5991443,1,23662849,11,4705135,1238,1.32,1.03,NaN,NaN,NaN
5398738,2,21257171,10,4549769,32940,1.73,1.16,NaN,1.098612,1.098612
1383046,3,5334985,11,4717855,43066,1.05,0.78,NaN,NaN,NaN
1110570,4,4150753,11,966351,1277,1.68,NaN,NaN,NaN,NaN


In [35]:
# predict week 10 data
X_test_wk10 = xgb.DMatrix(test[test['Week_num'] == 10].iloc[:, 3:], enable_categorical=True)

predictions_10 = model.predict(X_test_wk10)


# add week 10 prediction to test
test['wk_10_pred_dem'] = np.nan
test.loc[test['Week_num'] == 10, 'wk_10_pred_dem'] = predictions_10


# predict week 11 data
test_wk11 = test[test['Week_num'] == 11].copy(deep=True)

test_wk11.drop(['lag_3'], axis=1, inplace=True)
test_wk11.rename(columns={'lag_2': 'lag_3', 'lag_1': 'lag_2', 'wk_10_pred_dem': 'lag_1'}, inplace=True)

X_test_wk11 = xgb.DMatrix(test_wk11.iloc[:, 3:], enable_categorical=True)

predictions_11 = model.predict(X_test_wk11)


# add week 11 prediction to test
test['wk_11_pred_dem'] = np.nan
test.loc[test['Week_num'] == 11, 'wk_11_pred_dem'] = predictions_11


test.reset_index(drop=True, inplace=True)
test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1,wk_10_pred_dem,wk_11_pred_dem
0,0,25973294,11,4639078,35305,1.21,1.67,NaN,NaN,1.609438,NaN,1.223923
1,1,23662849,11,4705135,1238,1.32,1.03,NaN,NaN,NaN,NaN,1.204228
2,2,21257171,10,4549769,32940,1.73,1.16,NaN,1.098612,1.098612,1.300586,NaN
3,3,5334985,11,4717855,43066,1.05,0.78,NaN,NaN,NaN,NaN,0.981907
4,4,4150753,11,966351,1277,1.68,NaN,NaN,NaN,NaN,NaN,1.403921


In [36]:
submission = pd.DataFrame()

submission['id'] = np.arange(len(test))
submission['Demanda_uni_equil'] = test['wk_10_pred_dem'].combine_first(test['wk_11_pred_dem'])
submission['Demanda_uni_equil'] = np.expm1(submission['Demanda_uni_equil'])
submission.loc[submission['Demanda_uni_equil'] < 0, 'Demanda_uni_equil'] = 0

submission.head()

,id,Demanda_uni_equil
0,0,2.400501
1,1,2.334185
2,2,2.671447
3,3,1.669542
4,4,3.071133


In [37]:
submission.to_csv("xgb_prediction_7_log_dem.csv", index=False)